In [ ]:
from openpyxl import load_workbook

def parse_xlsx(filename, sheetname):
    wb = load_workbook(filename=filename, read_only=True)
    ws = wb[sheetname]

    sheet_rows = list()
    for num, row in enumerate(ws.rows):
        if num == 0:
            headers = [i.value for i in row]
        else:
            row_cells = [i.value for i in row]
            row_dict = dict(zip(headers, row_cells))
            sheet_rows.append(row_dict)
    return sheet_rows

In [ ]:
borrowing_requests = parse_xlsx('Feb13MikeSource/ILL Borrowing Requests (Loan) - 1-1-2016 - 12-31-2018 - Patron Recommendations.xlsx', 'Sheet')
# illrecpur = parse_xlsx('Feb13MikeSource/ILLRECPUR FY16 thru FY18 with usage thru 12-20-2018.xlsx', 'output_gjzj3985083270205064418') 

In [ ]:
borrowing_requests = [i for i in borrowing_requests
                     if set(i.values()) != {None,}]

## Find exact complete matching rows

In [ ]:
for i in borrowing_requests:
    if isinstance(i['Creation Date'], str):
        continue
    try:
        date = i['Creation Date']
        formatted_date = f"{date.month}/{date.day}/{date.year}"
        i['Creation Date'] = formatted_date
    except:
        print(i)

In [ ]:
exact_doubles = list()
already_in_exact_doubles = list()
others = list()

temp_found_so_far = list()

for row in borrowing_requests:
    if row in temp_found_so_far:
        if row not in exact_doubles:
            exact_doubles.append(row)
        else:
            already_in_exact_doubles.append(row)
    else:
        temp_found_so_far.append(row)
        
for row in temp_found_so_far:
    if row in exact_doubles:
        already_in_exact_doubles.append(row)
    else:
        if row not in others:
            others.append(row)

In [ ]:
print(len(borrowing_requests), len(exact_doubles), len(already_in_exact_doubles), len(others), len(temp_found_so_far))

In [ ]:
for i in exact_doubles:
    print(i)
    break

## Find those that match Author/Title

In [ ]:
temp_done_already = set()
temp_done_already_full_record = list()
author_title_matches = list()
not_author_title_match = list()

already_in_author_title_matches = list()
others2 = list()

for source_item in others:
    source_author, source_title = source_item['Loan Author'], source_item['Loan Title']
    if not temp_done_already:
        temp_done_already.add((source_author, source_title))
        temp_done_already_full_record.append(source_item)
        continue
    if (source_author, source_title) in temp_done_already:
        author_title_matches.append(source_item)
    else:
        temp_done_already.add((source_author, source_title))
        temp_done_already_full_record.append(source_item)
        
for row in temp_done_already_full_record:
    for match_item in author_title_matches:
        if (row['Loan Author'], row['Loan Title']) == (match_item['Loan Author'], match_item['Loan Title']):
            already_in_author_title_matches.append(row)
            break
    else:
        for others2_item in others2:
            if (row['Loan Author'], row['Loan Title']) == (others2_item['Loan Author'], others2_item['Loan Title']):
                break
        else:
            others2.append(row)

In [ ]:
len(others), len(author_title_matches), len(already_in_author_title_matches), len(others2), len(temp_done_already)

In [ ]:
for i in others2:
    print(i)
    break

## Find the fuzzy matches

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def make_matches(parsed_sheet1, parsed_sheet2):
    possible_matches = list()
    for num, sheet1_row in enumerate(parsed_sheet1):
        sheet1_row_title = sheet1_row['title']
        for sheet2_row in parsed_sheet2:
            if sheet2_row == sheet1_row:
                continue
            sheet2_row_title = sheet2_row['Loan Title']
            title_ratio = fuzz.token_sort_ratio(sheet1_row_title, sheet2_row_title)
            title_ratios.append(title_ratio)
            if title_ratio > 75:
                possible_matches.append((title_ratio, sheet1_row, sheet2_row))
    return possible_matches

In [ ]:
fuzzy_matches = list()
temp_these_done = set()
temp_these_done_full_record = list()
not_fuzzy_matches = list()

already_in_fuzzy_matches = list()
others3 = list()

title_ratios = list()

fuzzy_matches = make_matches(others2, others2)

In [ ]:
for ratio, a, b in fuzzy_matches:
    print(ratio)
    print(a['Loan Title'])
    print(b['Loan Title'])

In [ ]:
for i in range(10):
    print(len([n for n in title_ratios if i * 10 < n < 10 + i * 10]))

In [ ]:
import csv

def write_dict_to_csv(output_filename, a_list_of_dicts):
    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = a_list_of_dicts[0].keys()
        spamwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        spamwriter.writeheader()
        for row in a_list_of_dicts:
            spamwriter.writerow(row)

In [ ]:
write_dict_to_csv('Feb13MikeSource/FullMatches.csv', exact_doubles)
write_dict_to_csv('Feb13MikeSource/AuthorTitleMatches.csv', author_title_matches)
write_dict_to_csv('Feb13MikeSource/Other2.csv', others2)

In [ ]:
flattened_fuzzy_matches = list()
for a, b, c in fuzzy_matches:
    flattened_fuzzy_matches.extend([b, c])

## Export fuzzy matches to spreadsheet

  manually delete the non-matching pair rows
  
  convert the csv back into a list of dicts

In [ ]:
# This file is meant to be user-revised -- so do generate it twice.
# write_dict_to_csv('Feb13MikeSource/FuzzyMatches.csv', flattened_fuzzy_matches)

In [ ]:
import csv

hand_filtered_fuzzy_matches = list()

with open('Feb13MikeSource/FuzzyMatches.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for num, row in enumerate(reader):
        if num == 0:
            headers = row
            continue
        row_dict = dict(zip(headers, row))
#         if row_dict not in hand_filtered_fuzzy_matches:
        hand_filtered_fuzzy_matches.append(row_dict)

In [ ]:
hand_filtered_fuzzy_matches[0:2]

In [ ]:
merged_fuzzy_matches = list()

def merge_fuzzy_copies(a, b):
    a_trans, b_trans = int(a['Transaction Number']), int(b['Transaction Number'])
    if not merged_fuzzy_matches:
        merged_fuzzy_matches.append([a_trans, b_trans])
        return
    if 5303239 in (a_trans, b_trans):
        print(a_trans, b_trans)
    for i in merged_fuzzy_matches:
        if a_trans in i and b_trans in i:
            break
        elif a_trans in i:
            i.append(b_trans)
            break
        elif b_trans in i:         
            i.append(a_trans)
            break
    else:
        merged_fuzzy_matches.append([a_trans, b_trans])
#     print(merged_fuzzy_matches)

In [ ]:
for num, i in enumerate(hand_filtered_fuzzy_matches):
    if num % 2 != 0:
        a = hand_filtered_fuzzy_matches[num-1]
        b = hand_filtered_fuzzy_matches[num]
        merge_fuzzy_copies(a, b)

In [ ]:
len(merged_fuzzy_matches)

In [ ]:
# print(merged_fuzzy_matches)

In [ ]:
for i in merged_fuzzy_matches:
    if len(i) > 2:
        print(i)

## Find matches in ILLRECPUR


In [ ]:
illrecpur = parse_xlsx('Feb13MikeSource/ILLRECPUR FY16 thru FY18 with usage thru 12-20-2018.xlsx', 'output_gjzj3985083270205064418') 

In [ ]:
all_flexkeys = set()

duplicate_items = list()

for row in illrecpur:
    flexkey = row['flexkey']
    if flexkey in all_flexkeys:
        if row not in duplicate_items:
            duplicate_items.append(row)
    all_flexkeys.add(flexkey)

In [ ]:
len(duplicate_items)

In [ ]:
duplicate_flexkeys = set(i['flexkey'] for i in duplicate_items)

In [ ]:
len(duplicate_flexkeys)

In [ ]:
write_dict_to_csv('Feb13MikeSource/duplicate_illrecpur.csv', duplicate_items)


In [ ]:
counts_dict = dict()

for flexkey in duplicate_flexkeys:
    counts = 0
    for row in illrecpur:
        if row['flexkey'] == flexkey:
            counts += 1
    counts_dict[flexkey] = counts

In [ ]:
deduped_illrecpur = list()
dupes_done = set()

for row in illrecpur:
    row_flexkey = row['flexkey']
    if row_flexkey not in duplicate_flexkeys:
        row['duplicate counts'] = 1
        deduped_illrecpur.append(row)
    else:
        if row_flexkey not in dupes_done:
            row['duplicate counts'] = counts_dict[row_flexkey]
            deduped_illrecpur.append(row)
            dupes_done.add(row_flexkey)

In [ ]:
len(deduped_illrecpur)

In [ ]:
len(illrecpur)

In [ ]:
write_dict_to_csv('Feb13MikeSource/CountsDuplicatesILLRECPUR.csv', deduped_illrecpur)

In [ ]:
reduced_illrecpur = [i for i in deduped_illrecpur if int(i['duplicate counts']) > 1]

In [ ]:
len(reduced_illrecpur)

In [ ]:
write_dict_to_csv('Feb13MikeSource/ReducedDuplicatesILLRECPUR.csv', reduced_illrecpur)

In [ ]:
title_ratios = list()

def make_matches(parsed_sheet1, parsed_sheet2):
    possible_matches = list()
    for num, sheet1_row in enumerate(parsed_sheet1):
        sheet1_row_title = sheet1_row['title']
        for sheet2_row in parsed_sheet2:
            if sheet2_row == sheet1_row:
                continue
            sheet2_row_title = sheet2_row['Loan Title']
            title_ratio = fuzz.token_sort_ratio(sheet1_row_title, sheet2_row_title)
            title_ratios.append(title_ratio)
            if title_ratio > 75:
                possible_matches.append((title_ratio, sheet1_row, sheet2_row))
    return possible_matches



In [ ]:
illrecpur_to_borrowing_requests_possible_matches = make_matches(reduced_illrecpur, borrowing_requests)

In [ ]:
illrecpur_to_borrowing_requests_possible_matches

In [ ]:
writable_ill_brpm = []
for a, b, c in illrecpur_to_borrowing_requests_possible_matches:
    c['requested by']
    writable_ill_brpm.extend([c])

In [ ]:
write_dict_to_csv('Feb13MikeSource/illrecpur_to_borrowing_requests_matches2.csv', writable_ill_brpm)